In [20]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import json
import os
from dataclasses import dataclass, asdict

# Job object for ease of use
@dataclass(frozen=True)
class Job():
    title: str
    location: str

def GetAmountOfJobs():
    # The linkedin URL that we want to search through
    url = f"https://www.linkedin.com/jobs/search/?geoId=100902529&keywords=data&location=Flemish%20Region%2C%20Belgium"

    # Options
    chrome_options = Options()
    chrome_options.add_argument("--headless") # So the window doesn't pop up

    # Service
    service = Service('./chromedriver')

    # using chrome cause it's popular
    retval = 0
    with webdriver.Chrome(service=service, options=chrome_options) as driver:
        driver.get(url)
        html = driver.page_source

        # Beautiful soup, such beauty
        soup = BeautifulSoup(html, "html.parser")
        # Look the HTML up on the Chrome browser since it's completely different in Opera
        amount = soup.find('span', {'class', 'results-context-header__job-count'})
        
        retval = int(amount.text.split('+')[0].replace(',', ''))

    return retval

def GetJobData(start: int):
    # The linkedin URL that we want to search through
    url = f"https://www.linkedin.com/jobs/search/?geoId=100902529&keywords=data&location=Flemish%20Region%2C%20Belgium&start={start}"

    # Options
    chrome_options = Options()
    chrome_options.add_argument("--headless") # So the window doesn't pop up

    # Service
    service = Service('./chromedriver')

    # using chrome cause it's popular
    retval = []
    with webdriver.Chrome(service=service, options=chrome_options) as driver:
        driver.get(url)
        html = driver.page_source

        # Beautiful soup, such beauty
        soup = BeautifulSoup(html, "html.parser")
        # Look the HTML up on the Chrome browser since it's completely different in Opera
        jobs = soup.find_all('div', {'class', 'base-search-card__info'})

        # Get the text we want and write it to a file
        for job in jobs:
            title = job.find('h3', {'class', 'base-search-card__title'}).text.strip()
            metadata = job.find('div', {'class', 'base-search-card__metadata'})
            location = metadata.find('span', {'class', 'job-search-card__location'}).text.strip()
            retval.append(Job(title, location))

    # return list
    return retval

# Remove file if it already exists
if os.path.exists('jobs.json'):
    os.remove('jobs.json')

# Main thing to do
jobs = []
for start in range(0, GetAmountOfJobs(), 25):
    jobs += GetJobData(start)

txt_json = json.dumps([asdict(job) for job in jobs])
with open('jobs.json', 'a') as f:
    f.write(txt_json)

In [18]:
import re
# The linkedin URL that we want to search through
url = f"https://www.linkedin.com/jobs/search/?keywords=data&location=Flemish%20Region%2C%20Belgium"

# Options
chrome_options = Options()
chrome_options.add_argument("--headless") # So the window doesn't pop up

# Service
service = Service('./chromedriver')

# using chrome cause it's popular
with webdriver.Chrome(service=service, options=chrome_options) as driver:
    driver.get(url)
    html = driver.page_source

    # Beautiful soup, such beauty
    soup = BeautifulSoup(html, "html.parser")
    # Look the HTML up on the Chrome browser since it's completely different in Opera
    jobs = soup.find('span', {'class', 'results-context-header__job-count'})
print(int(jobs.text.split('+')[0].replace(',', '')))
#print(int(re.search("^(.+?)\+", jobs.text)).string)

7000
